In [ ]:
from pandarallel import pandarallel
import pandas as pd
import numpy as np
import csv
import re
import json
from sklearn.model_selection import train_test_split

from keras.models import Sequential
from keras.preprocessing import sequence
from keras.layers import Dropout
from keras.layers import  Dense, Embedding, LSTM, Bidirectional

import tensorflow as tf

np.random.seed(500)
pandarallel.initialize(nb_workers=6)


In [ ]:
df = pd.read_csv('../../data/processed/NOTEEVENTS_ML_DATASET.csv', nrows=10000, dtype=object)

In [ ]:
def drug_order_normalize(text):

    clinical_text = eval(text)
    drug_order = list(set(clinical_text))
    drug_order = list(filter(lambda x: len(x) >= 5, drug_order))
    drug_order_text = "".join(drug_order)
    return drug_order_text


In [ ]:
df['DRUG_ORDER'] = df['TEXT_NORMALIZED_ARRAY'].apply(drug_order_normalize)

In [ ]:
def convert_to_ascii(text):
    return [ord(char) for char in text]

x = df['DRUG_ORDER'].to_list()
y = df['LABELS'].to_list()

x = list(map(convert_to_ascii, x))
y = list(map(lambda x: eval(x)[:5], y))

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(
        x, y, test_size=0.3, random_state=100)

In [ ]:
max_len = 10000
x_train = sequence.pad_sequences(x_train, maxlen=max_len)
x_test = sequence.pad_sequences(x_test, maxlen=max_len)
y_test = np.array(y_test)
y_train = np.array(y_train)

In [ ]:
n_unique_words = 200
model = Sequential()
model.add(Embedding(n_unique_words, 128, input_length=max_len))
model.add(LSTM(64))
model.add(Dropout(0.5))
model.add(Dense(5, activation=tf.nn.tanh))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
history=model.fit(x_train, y_train,
           batch_size=32,
           epochs=12,
           validation_data=[x_test, y_test])
print(history.history['loss'])
print(history.history['accuracy'])